In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [2]:
import warnings
from pathlib import Path

import torch
from torch import nn
import pandas as pd

from attention import models
from attention import utils
from attention.vectorizer import Vectorizer
from attention.constants import ENGLISH, FRENCH, SEQ_SIZE, DECODER_INPUT, ENCODER_INPUT


warnings.filterwarnings('ignore')

SOURCE_DIR = Path('../')
DATA_DIR = SOURCE_DIR / 'data'
translation_fp = DATA_DIR / 'eng-fra.txt'

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Data

In [3]:
# vectorizer code
from attention.data import load_sentences_dataframe, assign_rows_to_split, TranslationDataset, generate_batches
from attention.vectorizer import Vectorizer

df = load_sentences_dataframe(translation_fp)
df = assign_rows_to_split(df, train_ratio=0.9, valid_ratio=0.05, test_ratio=0.05)

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)
df[ENGLISH] = df[ENGLISH].str.lower()
df = df[df[ENGLISH].str.startswith(eng_prefixes)]

dataset = TranslationDataset.from_dataframe(df)

# Source code

In [55]:
# model
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, embedding_size, embedding_matrix=None):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        # embedding layer
        if embedding_matrix is None:
            self.embedding = nn.Embedding(input_size, embedding_size)
        else:
            embedding_size = embedding_matrix.shape[-1]
            self.embedding = nn.Embedding.from_pretrained(embedding_matrix)
            
        
        # rnn layer
        self.gru = nn.GRU(embedding_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)
    

class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, embedding_size=None, embedding_matrix=None):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size

        # embedding layer
        if embedding_matrix is None:
            self.embedding = nn.Embedding(output_size, embedding_size)
        else:
            embedding_size = embedding_matrix.shape[-1]
            self.embedding = nn.Embedding.from_pretrained(embedding_matrix, )
            
        # rnn layer
        self.gru = nn.GRU(embedding_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = torch.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

    
class SimpleRnn(nn.Module):
    def __init__(self, encoder, decoder):
        super(SimpleRnn, self).__init__()

        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, input_sentence, output_sentence):
        hidden = self.encoder.initHidden()
        encoder_outputs = torch.zeros(1, SEQ_SIZE, self.encoder.hidden_size)
        
        for idx in range(SEQ_SIZE-1):
            token = input_sentence[idx]
            output, hidden = self.encoder(token, hidden)
            encoder_outputs[0, idx] = output[-1]

        decoder_outputs = torch.zeros(1, SEQ_SIZE-1, self.decoder.output_size)
        for idx in range(SEQ_SIZE-1):
            token = output_sentence[idx]
            output, hidden = self.decoder(token, hidden)
            decoder_outputs[0, idx] = output[-1]
            
        return decoder_outputs, hidden
    
def train_one_sent(sent, translation, model, optim):
    optim.zero_grad()
    decoder_outputs, hidden = model(sent, translation)
    criterion = nn.CrossEntropyLoss()
    loss = criterion(decoder_outputs.squeeze(0), translation_sentence[1:])
    loss.backward()
    optim.step()
    return loss.item()


# translate
def translate(sentence, model, vectorizer, device):
    vectorized = vectorizer.vectorize_sentence(sentence, language=ENGLISH).to(device)
    french_vocab = vectorizer.french_vocab
    
    encoder_hidden = encoder.initHidden()
    for idx in range(SEQ_SIZE):
        token = vectorized[idx]
        _, encoder_hidden = encoder_output, encoder_hidden = encoder(token, encoder_hidden)

    decoder_outputs = torch.zeros(1, SEQ_SIZE-1, decoder.output_size)

    for idx in range(SEQ_SIZE-1):
        token = vectorized[idx]
        output, hidden =  decoder(token, encoder_hidden)
        decoder_outputs[0, idx] = output[-1]

    
    indices = torch.argmax(decoder_outputs, dim=-1).squeeze(0)
    words = [french_vocab.lookup_index(idx) for idx in indices]
    print(' '.join(words))

# Embeddings

## Fasttext

In [48]:
import fasttext.util

fasttext.util.download_model('en', if_exists='ignore')
fasttext.util.download_model('fr', if_exists='ignore')

ft_english = fasttext.load_model('cc.en.300.bin')
ft_french = fasttext.load_model('cc.fr.300.bin')

vectorizer = dataset.vectorizer
english_embedding_matrix = vectorizer.build_embedding_matrix_from_fasttext(ft_english, lang=ENGLISH)
french_embedding_matrix = vectorizer.build_embedding_matrix_from_fasttext(ft_french, lang=FRENCH)

## spacy

In [ ]:
import spacy

#!python -m spacy download fr_core_news_md
#!python -m spacy download en_core_web_sm

spacy_english = spacy.load('en_core_web_sm')
spacy_french = spacy.load('fr_core_news_md')

vectorizer = dataset.vectorizer
english_embedding_matrix = vectorizer.build_embedding_matrix_from_spacy(spacy_english, lang=ENGLISH)
french_embedding_matrix = vectorizer.build_embedding_matrix_from_spacy(spacy_french, lang=FRENCH)

# Model Construction

In [ ]:
# Model params
use_pretrained_embeddings = False
hidden_size = 256
embedding_size = 300
english_vocab_size = len(dataset.vectorizer.english_vocab)
french_vocab_size = len(dataset.vectorizer.french_vocab)

# use pretrained embeddings?
if use_pretrained_embeddings:
    encoder = EncoderRNN(english_vocab_size, hidden_size, embedding_matrix=english_embedding_matrix)
    decoder = DecoderRNN(hidden_size, french_vocab_size, embedding_matrix=french_embedding_matrix)
else:
    encoder = EncoderRNN(english_vocab_size, hidden_size, embedding_size)
    decoder = DecoderRNN(hidden_size, french_vocab_size, embedding_size)


# TRAIN
model = SimpleRnn(encoder, decoder).to(device)

# Train

In [80]:
# training params
nb_epochs = 10
learning_rate = 0.001

# create optimizer
optim = torch.optim.Adam(model.parameters(), learning_rate)

# run epochs
for epoch in range(nb_epochs):
    epoch_losses = []
    for batch in generate_batches(dataset, batch_size=1):
        single_sentence, translation_sentence = batch[ENCODER_INPUT], batch[DECODER_INPUT]
        single_sentence = single_sentence.squeeze(0)
        translation_sentence = translation_sentence.squeeze(0)
        loss = train_one_sent(
            single_sentence, 
            translation_sentence, 
            model,
            optim
        )
        epoch_losses.append(loss)
    avg_loss = sum(epoch_losses) / len(epoch_losses)
    print(f'loss at {epoch+1} epoch is {avg_loss:.2f}')

loss at 1 epoch is 3.01
loss at 2 epoch is 1.98
loss at 3 epoch is 1.34
loss at 4 epoch is 0.90
loss at 5 epoch is 0.62
loss at 6 epoch is 0.45
loss at 7 epoch is 0.36
loss at 8 epoch is 0.30
loss at 9 epoch is 0.27
loss at 10 epoch is 0.25


# Inference (Translation)

In [83]:
sentences = [
    "you are so childish sometimes",
    "i am too short.",
    "he is unpopular for some reason",
    "she is french."
    
    "this is a new sentence",
    "I dont like this.",
    "I love studying math",
    "he went to school"
]
for sent in sentences:
    translate(sent, model, vectorizer)

vous es parfois si puerile . <pad> <pad> <pad>
je trop jeune trop est je <pad> <pad> <pad>
il il des il il il il <pad> il
elle n elle elle elle elle elle elle elle
il elle une une il <eos> <pad> <pad> <pad>
je je lisent en je <pad> <pad> <pad> <pad>
il il il il il <pad> il il il


In [72]:
df.head()

,english,french,split
77646,you are so childish sometimes.,Tu es parfois si puérile.,train
39791,she is in need of help.,Elle a besoin d'aide.,train
6496,i am too short.,Je suis trop petit.,train
83279,he is unpopular for some reason.,Il est impopulaire pour une raison quelconque.,train
5278,she is french.,Elle est française.,train
